In [1]:
using CSV, DataFrames, Statistics, HTTP

In [2]:
url = "https://raw.githubusercontent.com/ArieBeresteanu/Bounds.jl/main/Fortran/Chtdata.csv"

http_response = HTTP.get(url)
file = CSV.File(http_response.body,header=false)

cht = DataFrame(file)

rename!(cht, :Column1 => :yl, :Column2 => :yu, :Column3 => :edu)
first(cht,3)

,yl,yu,edu
,Float64,Float64,Int64
1,4.0,4.0969,28
2,4.5441,4.699,28
3,4.1761,4.301,34


In [3]:
s = describe(cht, :mean, cols=[:yl,:yu])
H0 = [s[1,2], s[2,2]]

2-element Vector{Float64}:
 4.434663145221974
 4.967426358164033

In [6]:
include("C:\\Users\\The Group Leader\\Notebooks\\Research\\Bounds.jl-1\\src\\setBLP.jl")

LoadError: LoadError: UndefVarError: vector not defined
in expression starting at C:\Users\The Group Leader\Notebooks\Research\Bounds.jl-1\src\setBLP.jl:1

In [5]:
setBLP.EY(cht[!,:yl],cht[!,:yu],H0)

LoadError: UndefVarError: EY not defined